# Solving your Bayesian dreams with `bilby` 

<img src="bilby.jpg" alt="drawing" width="400"/>

### What is `bilby`?

A pure `Python`-based software package also known as the Bayesian Inference LiBrarY ([Ashton et al., 2019](https://ui.adsabs.harvard.edu/abs/2019ApJS..241...27A/abstract); [Romero-Shaw et al., 2020](https://ui.adsabs.harvard.edu/abs/2020MNRAS.499.3295R/abstract))

Was originally developed solely as a user-friendly replacement for [`LALInference`](https://lscsoft.docs.ligo.org/lalsuite/lalinference/index.html), the parameter estimation package developed by LIGO for analysis of gravitational-wave events. It was adopted as the official parameter estimation code of the LIGO-Virgo-KAGRA collaboration during the 3rd observing run, and was used to analyse some of the most unusual gravitational-wave events detected to date, such as  [GW190425](https://ui.adsabs.harvard.edu/abs/2020ApJ...892L...3A/abstract), [GW190521](https://ui.adsabs.harvard.edu/abs/2020PhRvL.125j1102A/abstract) and [GW190814](https://ui.adsabs.harvard.edu/abs/2020ApJ...896L..44A/abstract).

During early development it was soon realised the core functionality of `bilby` was general enough that it warrented compartmentalising the package so that the base functions (`bilby.core`) could be imported and utilised separately from the gravitational-wave specific components (`bilby.gw`). Most works outside of gravitational-wave astronomy that have utilised `bilby` tend use it as a user-friendly interface to several MCMC and nested sampling algorithms, such as the well-known affine-invariant MCMC sampler in Dan Foreman-Mackey's [`emcee`](https://github.com/dfm/emcee) package and the dynamic nested sampler built into [`dynesty`](https://github.com/joshspeagle/dynesty/).

### Installing `bilby`

First, you need to check whether you are using `Python3.7` or above. `Python2.7` will **not** work!

The easiest (and recommended) way to install `bilby` is via `pip`:
```
pip install bilby
```

Alternatively, you can clone the GitLab repository: [https://git.ligo.org/lscsoft/bilby](https://git.ligo.org/lscsoft/bilby).


### Useful Resources:

Here's some links to useful papers and documents in case you're interested in how the underlying mathematic works
* General introduction to Bayesian inference (slight focus on gravitational waves): [Thrane \& Talbot (2019)](https://ui.adsabs.harvard.edu/abs/2019PASA...36...10T/abstract).
* Mathematics behind nested sampling algorithms:[Skilling (2004)](https://aip.scitation.org/doi/abs/10.1063/1.1835238)
* The MultiNest implemtation of nested sampling: [Feroz, Hobson \& Bridges (2009)](https://ui.adsabs.harvard.edu/abs/2009MNRAS.398.1601F/abstract).

If you ever run into issues, you can reach out to the `bilby` support desk: [contact+lscsoft-bilby-1846-issue-@support.ligo.org](contact+lscsoft-bilby-1846-issue-@support.ligo.org).

Join the `bilby` Slack: [https://bilby-code.slack.com/](https://bilby-code.slack.com/).

Or even ask `bilby`-tagged questions on [Stack Overflow](https://stackoverflow.com/questions/tagged/bilby)!


# A quick primer: What is the goal of Bayesian parameter estimation?

We want to know the probabilty that some model ($\mu$) definied by some parameters ($\theta$) matches some data ($d$).

This can be done by evaluating Bayes' theorem:
### $$ p(\theta | d) = \frac{\mathcal{L}(d | \theta) \pi(\theta)}{\mathcal{Z}(d)} $$

Here, $p(\theta | d)$ is the posterior probability that the model parameters accurately describe the data, $\mathcal{L}(d | \theta)$ is the likelihood of the data given the chosen model parameters, $\pi(\theta)$ is our prior knowledge of what the model parameters should be, and $\mathcal{Z}(d)$ is the Bayesian evidence (can ignore this for now...)

Calculating the posterior probability distribution directly, especially if we have many model parameters, is difficult. We instead tend to generate *posterior samples* by using smart, stochastic sampling algorithms (e.g. MCMC and nested sampling) that explore all possible values of our model parameters and trend towards those with the highest probability of matching the data.

# Getting started

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import bilby

In [ ]:
bilby?

## Model: Simple linear slope $\rightarrow$ $ y = ax + c $

In [ ]:
def line(x, a, c):
    """ A simple linear slope """
    return (a * x) + c

## Making some fake data 

In [ ]:
np.random.seed(123)

n = 10

x = np.linspace(-10, 10, n)
y = line(x, 0.5, 1)

y_errs = 0.1 + 1.0 * np.random.rand(n)

noise = np.random.uniform(-1.5, 1.5, n)

data = y + noise

plt.errorbar(x, data, yerr=y_errs, fmt="o")
plt.plot(x, y, "--")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

# Using `bilby` to model the data

## Example 1. Using the built-in Gaussian likelihood function

In order to fit the data, we first need to define our likelihood function. 

The Gaussian likelihood is often used for this, where the residuals after subtracting our model would (ideally) look like Gaussian noise:

### $$ \mathcal{L}(d | \theta) = \prod_{i}^{N} \frac{1}{\sqrt{2 \pi \sigma^{2}}} \exp \Big[ -\frac{(d_{i} - \mu_{i}(\theta))^{2}}{2\sigma^{2}} \Big] $$

As implied by the name, the likelihood function tells us how *likely* it is that a given iteration of our model accurately matches the data.

For this particular example, we're going to use the pre-built Gaussian likelihood in `bilby`, where the variance ($\sigma$ in the above equation) is set to be our uncertainties on the data. This is called using `bilby.likelihood.GaussianLikelihood`.

In [ ]:
bilby.likelihood.GaussianLikelihood?

In [ ]:
likelihood = bilby.likelihood.GaussianLikelihood(x=x,y=data, func=line, sigma=y_errs)

### Setting the priors

Next step in setting up our parameter estimation is setting up the priors. 

Priors are important, as they represent our pre-existing knowledge of what the model parameters should be. This includes things such as the allowed range of values we want to search over and whether some values are more probable than others. 

For the fake data we generated above, we're going to assume that we know nothing about the probability that one set of parameters matches the data better than another. Hence we assume *Uniform* priors between some upper and lower bounds

### $$ \pi(\theta) = 1,\,\, {\rm where} \,\, \theta_{\rm low} < \theta < \theta_{\rm upp} $$

The priors are defined as a `Python` dictionary (`priors = dict()`), where each dictionary key corresponds to one of our model parameters. E.g. `priors["a"] = bilby.prior.Uniform(0, 1, "a")`.

In [ ]:
priors = dict()
priors["a"] = bilby.prior.Uniform(0, 1, "a")
priors["c"] = bilby.prior.Uniform(0, 4, "c")

### Running the parameter estimaiton

Now that we've got our likelihood function and priors all set up, it's time to start sampling the posterior distributions!

To do this, we're going to run `bilby` using the default sampler, `dynesty`, which uses a nested sampling algorithm to explore the parameter space. This is run via `bilby.run_sampler`.

We will tell `dynesty` to explore the parameter space using 512 "live points". The more live points that are used, the more densly the parameter space will be explored, at the expense of the sampling taking much longer to converge to the most probable region.

In [ ]:
bilby.run_sampler?

In [ ]:
result = bilby.run_sampler(likelihood=likelihood, priors=priors, sampler="dynesty", nlive=512, 
                           label="line_test", outdir="output")

### Looking at the results

Once the sampling is complete, we can look at the resulting posterior probability distributions by running `result.plot_corner` to make a "corner" or "triangle" plot. 
These display the one- and two-dimensional posterior probability distributions in a way that makes it (relatively) easy to assess the level of covariance between individual model parameter.

Parameters that are highly covariant will have 2-D posteriors that appear stretched or distorted (also known as having a "degenerate" solution), while uncorrelated parameters will look like round blobs.

The orange lines and square correspond to the original "injected" or "true" values of `a` and `c` that were used to generate the fake data.

In [ ]:
# Specify the "true" values of a & c
truths = dict(a=0.5, c=1.0)

# Plot the 1- and 2-D posteriors alongside the true values
result.plot_corner(truths=truths, dpi=150)

### Posterior probability check

So we've looked at the posterior distributions, and they (hopefully) look nice and well converged around the injected values of `a` and `c`. The next thing we can do is extract the median a-posteriori solution as well as a bunch of random draws from the posteriors and see how the resulting models compare against the data.

We do this by first extracting the posterior samples for `a` and `c` using `result.posterior["a"].values` and `result.posterior["b"].values`. 

Note, that the posteriors samples in `result.posteriors` are saved to a `pandas` dataframe that can be queried using other methods as well.

To plot the random draws from the posteriors, we just evaluate the model using random samples extracted from the `a` and `c`.

In [ ]:
a_vals = result.posterior["a"].values
c_vals = result.posterior["c"].values

In [ ]:
# Get the median fit
xfit = np.linspace(-12, 12, 32)
med_model = line(xfit, np.median(a_vals), np.median(c_vals))

# Plot the data
plt.errorbar(x, data, yerr=y_errs, fmt="o")

# Plot 25 random draws from the posteriors
for i in range(0, 25):
    instance = line(xfit, a_vals[i], c_vals[i])
    plt.plot(xfit, instance, lw=1, color="tab:orange", alpha=0.3)

# Plot the median fit & original injected slope:
plt.plot(xfit, med_model, ls="-", lw=2, color="k", label="Median fit")
plt.plot(x, y, ls="--", lw=2, color="tab:green", label="Truth")    
plt.xlim(-11, 11)
plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc="best")
plt.show()

### Getting the median values and uncertainties

To extract the median a-posteriori values and 68% credible intervals (can be thought of as a "1-sigma" uncertainty), we can use `result.get_one_dimensional_median_and_error_bar(key)`, where `key="a"` or `key="c"`. 

In [ ]:
a_result = result.get_one_dimensional_median_and_error_bar("a")
c_result = result.get_one_dimensional_median_and_error_bar("c")

print("Median and 68% C.I.")
print("a = {0} +{1}/-{2}".format(round(a_result.median,2), round(a_result.plus,2), round(a_result.minus,2)))
print("c = {0} +{1}/-{2}".format(round(c_result.median,2), round(c_result.plus,2), round(c_result.minus,2)))

---

## Example 2. Custom likelihood function

Sometimes, we don't want to just use the basic, built-in likelihood functions that `bilby` comes with.
In this example, we'll quickly go through how to use a custom made likelihood function that includes an extra "error in quadrature" (or EQUAD) term to account for possible underestimation of the uncertainties in our fake data.

In this case, we'll again use a Gaussian likelihood function,

### $$ \mathcal{L}(d | \theta) = \prod_{i}^{N} \frac{1}{\sqrt{2 \pi \sigma_{i}^{2}}} \exp \Big[ -\frac{(d_{i} - \mu_{i}(\theta))^{2}}{2\sigma_{i}^{2}} \Big], $$
but this time the values of $\sigma$ will be a little different.

Instead of just passing the uncertainties on our y-values, we will instead add in this extra EQUAD parameter ($\sigma_{Q}$) as 

### $$ \sigma_{i}^{2} = \sigma_{y,i}^{2} + \sigma_{Q}^{2}. $$

This will serve to inflate the uncertainties on our y-values by a small amount. 

Writing custom likelihood functions can be a little tricky at first, especially if you're unfamiliar with `Python` classes. So for this example, we're essentially re-writing the default Gaussian likelihood function in `bilby` to take this extra EQUAD parameter as well as our normal model and model parameter.

Note, this likelihood function will require some additional modifications if you want to use it with a more general model, instead of just our current `line` model.

In [ ]:
class CustomGaussianLikelihood(bilby.likelihood.Likelihood):
    def __init__(self, x, y, y_err, func, parameters):
        """
        Custom Gaussian Likelihood

        A modified version of the Gaussian likelihood for fitting
        straight lines. Also incorperates an EQUAD parameter.

        Parameters
        ----------
        x : ndarray
            Array of independent values.
        y : ndarray
            Array of dependent values.
        y_err : ndarray 
            Formal errors on the dependent values.
        func : function
            Function containing the model to be fit to the data.
        parameters : dict_like
            Dictionary of parameter keys.
        """
        
        super().__init__()
        self.x = x
        self.y = y
        self.y_err = y_err
        self.func = func
        self.parameters = parameters
        
    def log_likelihood(self):
        self.sigma = np.sqrt((self.y_err**2) + (self.parameters["sigma"]**2))
        
        self.model = self.func(self.x, self.parameters["a"], self.parameters["c"])
        self.residual = (self.y - self.model)**2

        ln_like = np.sum(- (self.residual / (2 * self.sigma**2))
            - np.log(2 * np.pi * self.sigma**2) / 2)

        return ln_like

### Running `bilby` with the custom likelihood

And now, we just initialise and then run `bilby` as before!

In [ ]:
# Initialise the parameters dictionary
parameters = dict(a=None, c=None, sigma=None)

# Initialise the likelihood function
likelihood = CustomGaussianLikelihood(x=x, y=y, y_err=y_errs, func=line, parameters=parameters)

# Set priors
priors = dict()
priors["a"] = bilby.prior.Uniform(0, 1, "a")
priors["c"] = bilby.prior.Uniform(0, 4, "c")
priors["sigma"] = bilby.prior.Uniform(0, 1, "sigma")

In [ ]:
result = bilby.run_sampler(likelihood=likelihood, priors=priors, sampler="dynesty", nlive=512, 
                           label="custom_priors", outdir="output")

### Inspecting the posterior distributions

In [ ]:
truths = dict(a=0.5, c=1.0, sigma=None)

result.plot_corner(truths=truths, dpi=150)

### Posterior predictive check

In [ ]:
# Extract the posteriors
a_vals = result.posterior["a"].values
c_vals = result.posterior["c"].values
sigmas = result.posterior["sigma"].values

In [ ]:
# Get the median model
xfit = np.linspace(-12, 12, 32)
med_model = line(xfit, np.median(a_vals), np.median(c_vals))

# Plot the data with errorbars inflated by the median EQUAD value 
plt.errorbar(x, data, yerr=np.sqrt(y_errs**2 + np.median(sigmas**2)), fmt="o")

# Plot 25 random draws from the posterior
for i in range(0, 25):
    instance = line(xfit, a_vals[i], c_vals[i])
    plt.plot(xfit, instance, lw=1, color="tab:orange", alpha=0.3)

# Plot the median fit & original injected slope:
plt.plot(xfit, med_model, ls="-", lw=2, color="k", label="Median fit")
plt.plot(x, y, ls="--", lw=2, color="tab:green", label="Truth")    
plt.xlim(-11, 11)
plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc="best")
plt.show()

In [ ]:
a_result = result.get_one_dimensional_median_and_error_bar("a")
c_result = result.get_one_dimensional_median_and_error_bar("c")
sigma_result = result.get_one_dimensional_median_and_error_bar("sigma")

print("Median and 68% C.I.")
print("a = {0} +{1}/-{2}".format(round(a_result.median,2), round(a_result.plus,2), round(a_result.minus,2)))
print("c = {0} +{1}/-{2}".format(round(c_result.median,2), round(c_result.plus,2), round(c_result.minus,2)))
print("sigma = {0} +{1}/-{2}".format(round(sigma_result.median,2), round(sigma_result.plus,2), 
                                     round(sigma_result.minus,2)))

### So how'd it do?

The median recovered values exactly match the initial values that we injected! However, the 68% credible intervals for both `a` and `c` are a little larger. 

This larger uncertainty makes sense though, as we've added an extra level of uncertainty into the data by including the EQUAD parameter. Therefore a slightly wider range of possible models can adequetely fit within the inflated data uncertainties.

---
## Example 3. Model selection

One of the most powerful applications of Bayesian inference is comparing whether two or more different models provide a better match to the data. 
This process, known as Bayesian model selection relies on that *Evidence* parameter, $\mathcal{Z}(d)$ from before.

In this case, we're going to re-fit our fake data from before using a quadratic function instead of a simple linear fit, and test whether it is the preferred model.

### Simple quadratic model: $y = ax^{2} + bx +c$

In [ ]:
def quadratic(x, a, b, c):
    """ A simple quadratic function """
    return (a * x**2) + (b * x) + c

### Setting up the likelihood & priors

As in Example 1, we're just going to use the pre-existing Gaussian likelihood that is built into `bilby`.

In [ ]:
# Initialise the likelihood function
likelihood = bilby.likelihood.GaussianLikelihood(x=x,y=data, func=quadratic, sigma=y_errs)

# Set-up the priors
priors = dict()
priors["a"] = bilby.prior.Uniform(-2, 2, "a")
priors["b"] = bilby.prior.Uniform(-2, 2, "b")
priors["c"] = bilby.prior.Uniform(-2, 2, "c")

In [ ]:
result_quad = bilby.run_sampler(likelihood=likelihood, priors=priors, sampler="dynesty", nlive=512, 
                                label="quadratic_test", outdir="output")

In [ ]:
# Plotting the 1- & 2-D posteriors
result_quad.plot_corner(dpi=150)

### Posterior predictive checks

In [ ]:
a_vals = result_quad.posterior["a"].values
b_vals = result_quad.posterior["b"].values
c_vals = result_quad.posterior["c"].values

In [ ]:
# Get the median model
xfit = np.linspace(-12, 12, 32)
med_model = quadratic(xfit, np.median(a_vals), np.median(b_vals), np.median(c_vals))

# Plot the data
plt.errorbar(x, data, yerr=y_errs, fmt="o")

# Plot 25 random draws from the posterior
for i in range(0, 25):
    instance = quadratic(xfit, a_vals[i], b_vals[i], c_vals[i])
    plt.plot(xfit, instance, lw=1, color="tab:orange", alpha=0.3)

    
# Plot the median fit & original injected slope:
plt.plot(xfit, med_model, ls="-", lw=2, color="k", label="Median fit")
plt.plot(x, y, ls="--", lw=2, color="tab:green", label="Truth")    
plt.xlim(-11, 11)
plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc="best")
plt.show()

In [ ]:
# Get the 1-D median and 68% upper & lower confidence intervals
a_result = result_quad.get_one_dimensional_median_and_error_bar("a")
b_result = result_quad.get_one_dimensional_median_and_error_bar("b")
c_result = result_quad.get_one_dimensional_median_and_error_bar("c")

print("Median and 68% C.I.")
print("a = {0} +{1}/-{2}".format(round(a_result.median,2), round(a_result.plus,2), round(a_result.minus,2)))
print("b = {0} +{1}/-{2}".format(round(b_result.median,2), round(b_result.plus,2), round(b_result.minus,2)))
print("c = {0} +{1}/-{2}".format(round(c_result.median,2), round(c_result.plus,2), round(c_result.minus,2)))

### Comparing the quadratic versus linear models

Just by looking at the posteriors and posterior predictive plots, it's pretty clear that the full quadratic model isn't fitting the data as well as our previous linear fits. 

However, to quantitatively determine which model best describes the data, we must first read in the results from our old linear fit using `bilby.result.read_in_result` as follows.

In [ ]:
result_line = bilby.result.read_in_result("./output/line_test_result.json")

### Computing the Bayes factor

Now that we have both sets of results read in, we can use the Bayesian evidences that were computed by `dynesty` to determine which model best describes the data. 

As a quick aside, the Bayesian evidence is simply the integral of our likelihood function over all values of our model parameters (sometimes referred to as the "fully marginalised" likelihood)
### $$ \mathcal{Z}(d) = \int d\theta \mathcal{L}(d | \theta) \pi(\theta) $$
Thankfully, `dynesty` provides the evidence as an output. So we don't have to try and compute this integral ourselves (which can be quite a painful process!).

Once we have the evidences for our two models, we can then compute the "Bayes factor" by dividing the evidence for model 1 by the evidence for model 2 (this is why the Bayes factor is sometimes referred to as the "ratio of evidences").
### $$ \mathcal{B}_{1/2} = \frac{\mathcal{Z}_{1}}{\mathcal{Z}_{2}} $$

One thing to note for our case is that we have computed the likelihood function in natural log space (i.e as a "log-likelihood"). This is mainly to avoid numerical precision issues that can arise when dealing with either very large or very small numbers. 
As a result, we extract the "log-evidences" from our `result` objects using `result.log_evidence`, and compute a "log Bayes factor" by subtracting the log-evidence of model 2 from model 1 as
### $$ \ln \mathcal{B}_{1/2} = \ln \mathcal{Z}_{1} - \ln \mathcal{Z}_{2} .$$


In [ ]:
# Get the log-evidences for the linear & quadratic fits
lnZ_line = result_line.log_evidence
lnZ_quad = result_quad.log_evidence

# Compute log Bayes factor
lnBF = lnZ_line - lnZ_quad

print("log Bayes factor = {0}".format(lnBF))

### Interpreting the Bayes factor

Exactly what makes a "good" Bayes factor threshold, i.e the minimum Bayes factor needed to say one model is more strongly preferred over another, is somewhat of a personal opinion. 

An often used interpretation comes from [Kass & Raferty (1995)](). Here, $\ln\mathcal{B} > 5$ means one model is considered *very strongly* preferred over the other, $3 < \ln\mathcal{B} < 5$ indicates the model is *moderately* preferred over the other, $1 < \ln\mathcal{B} < 3$ is a *weak* preference of one over the other, and $0 < \ln\mathcal{B} < 1$ is a marginal preference of one model over the other.

In the case of a marginal preference, then we often tend assume the simplest model provides an adequete represntation of the data and is therefore *preferred*.